## Pytorch training module

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# __init__ : define all module parameters
# forward() : returns the output of the model.

class model_number(nn.Module):
    def __init__(self):
        super().__init__()
        initial_theta_0 = torch.Tensor([1])
        initial_theta_1 = torch.Tensor([2])
        self.theta_0 = torch.nn.Parameter(initial_theta_0)
        self.theta_1 = torch.nn.Parameter(initial_theta_1)

    def forward(self, tensor_number_tasks):
        return self.theta_1 * tensor_number_tasks + self.theta_0

In [4]:
net = model_number()

In [5]:
list(net.parameters())

[Parameter containing:
 tensor([1.], requires_grad=True),
 Parameter containing:
 tensor([2.], requires_grad=True)]

In [6]:
list(net.named_parameters())

[('theta_0',
  Parameter containing:
  tensor([1.], requires_grad=True)),
 ('theta_1',
  Parameter containing:
  tensor([2.], requires_grad=True))]

In [7]:
tensor_number_tasks = torch.Tensor([3])
tensor_number_tasks

tensor([3.])

In [8]:
net(tensor_number_tasks) # same as net.forward(tensor_number_tasks)

tensor([7.], grad_fn=<AddBackward0>)

- Train linear regression with this model

In [9]:
# loss function
def compute_loss(list_number_tasks, list_number_questions, model_number_questions):
    mse_loss = torch.Tensor([0])

    for number_tasks, number_questions in zip(list_number_tasks, list_number_questions):
        # squared error for (number_tasks, number_questions)
        tensor_number_tasks = torch.Tensor([number_tasks])
        estimator_number_questions = model_number_questions(tensor_number_tasks)

        error = estimator_number_questions - number_questions
        squared_error = error * error

        mse_loss += squared_error     # adding computed error to the loss

    mse_loss /= len(list_number_tasks)  

    return mse_loss

In [10]:
# train
def train_linear_regression(list_number_tasks, 
                            list_number_questions, 
                            learning_rate=0.02, 
                            number_training_steps=100):

    net = model_number()

    optimiser = torch.optim.SGD(net.parameters(), lr=learning_rate)

    for _ in range(number_training_steps):
        optimiser.zero_grad()
        mse_loss = compute_loss(list_number_tasks, list_number_questions, model_number_questions=net)
        mse_loss.backward() 
        optimiser.step()

        print("loss --> ", mse_loss.item())

    print("Final Parameters --> ", list(net.named_parameters()))

    return net

In [11]:
list_number_tasks = torch.Tensor([[1.], [2.], [4.], [4.], [5.], [6.], [6.], [6.], [8.], [8.], [9.], [10.]])
list_number_questions = torch.Tensor([[5.], [11.], [21.], [22.], [26.], [31.], [32.], [31.], [41.], [42.], [48.], [52.]])

In [12]:
res = train_linear_regression(list_number_tasks=list_number_tasks,
                             list_number_questions=list_number_questions,
                             learning_rate=0.02,
                             number_training_steps=10)

loss -->  382.5
loss -->  152.1065673828125
loss -->  60.67946243286133
loss -->  24.397375106811523
loss -->  9.99804973602295
loss -->  4.282337665557861
loss -->  2.0124921798706055
loss -->  1.1100648641586304
loss -->  0.7502853870391846
loss -->  0.6058599948883057
Final Parameters -->  [('theta_0', Parameter containing:
tensor([1.3584], requires_grad=True)), ('theta_1', Parameter containing:
tensor([5.0112], requires_grad=True))]


- With linear operation in pytorch
  

In [13]:
# toch.nn.Linear
# in_features: Number of features as input to the linear operation. 
# out_features: Number of features as outputted by the linear operation.

In [16]:
class model_feature(nn.Module):
    def __init__(self):
        super().__init__()
        # self.linear is automatically initialised
        self.linear = torch.nn.Linear(in_features=1,
                                      out_features=1)

    def forward(self, tensor_number_tasks):
        return self.linear(tensor_number_tasks)

In [18]:
net_model = model_feature()
list(net_model.named_parameters())

[('linear.weight',
  Parameter containing:
  tensor([[-0.2497]], requires_grad=True)),
 ('linear.bias',
  Parameter containing:
  tensor([-0.8738], requires_grad=True))]

- TRAIN model with optimizer, loss function and linear operations

In [19]:
def linear_regression(tensor_number_tasks,
                      tensor_number_questions,
                      lr=0.02,
                      training_steps=200):

    net = model_feature()  
    loss = torch.nn.MSELoss()  

    optimiser = torch.optim.SGD(net.parameters(), lr=lr)

    for _ in range(training_steps):
        optimiser.zero_grad()
        estimator_number_questions = net(tensor_number_tasks)  # net.forward
        mse_loss = loss(input=estimator_number_questions,      # loss.forward
                        target=tensor_number_questions)
        mse_loss.backward()
        optimiser.step()
        print("loss --> ", mse_loss.item())

    print("Parameters --> ", list(net.named_parameters()))
    return net

In [20]:
# main()
tasks = [1, 2, 4, 4, 5, 6, 6, 6, 8, 8, 9, 10]
questions = [5, 11, 21, 22, 26, 31, 32, 31, 41, 42, 48, 52]

tensor_tasks = torch.Tensor(tasks).view(-1, 1)
tensor_questions = torch.Tensor(questions).view(-1, 1)
print(tensor_questions)
linear_regression(tensor_tasks, tensor_questions, lr=0.02, training_steps=10)

tensor([[ 5.],
        [11.],
        [21.],
        [22.],
        [26.],
        [31.],
        [32.],
        [31.],
        [41.],
        [42.],
        [48.],
        [52.]])
loss -->  1091.7763671875
loss -->  433.4320373535156
loss -->  172.1862335205078
loss -->  68.51801300048828
loss -->  27.380075454711914
loss -->  11.055561065673828
loss -->  4.577579498291016
loss -->  2.0069143772125244
loss -->  0.9867621064186096
loss -->  0.5818875432014465
Parameters -->  [('linear.weight', Parameter containing:
tensor([[5.1187]], requires_grad=True)), ('linear.bias', Parameter containing:
tensor([0.4701], requires_grad=True))]


model_feature(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)